In [1]:
import json
import pandas as pd
from datetime import datetime, timedelta

In [2]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.version import VERSION

def connect_to_snowflake():
  # Create Snowflake Session object
  print("Connecting to Snowflake...")
  connection_parameters = json.load(open('connection.json'))
  session = Session.builder.configs(connection_parameters).create()

  # Current Environment
  print("Current Environment...")
  snowflake_environment = session.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
  snowpark_version = VERSION
  print('   User                        : {}'.format(snowflake_environment[0][0]))
  print('   Role                        : {}'.format(snowflake_environment[0][1]))
  print('   Database                    : {}'.format(snowflake_environment[0][2]))
  print('   Schema                      : {}'.format(snowflake_environment[0][3]))
  print('   Warehouse                   : {}'.format(snowflake_environment[0][5]))
  print('   Snowflake version           : {}'.format(snowflake_environment[0][4]))
  print('   Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))
  return session
session = connect_to_snowflake()


Connecting to Snowflake...
Current Environment...
   User                        : TAKEHOME_AY
   Role                        : TAKE_HOME_USER2
   Database                    : TAKE_HOME_USER2
   Schema                      : None
   Warehouse                   : TRANSFORM_WH
   Snowflake version           : 7.10.2
   Snowpark for Python version : 1.2.0


In [ ]:
sensor_df = session.table("SENSOR.TIMESERIES")
sensor_pdf = sensor_df.to_pandas()
# readings_df = session.table("SENSOR.READINGS")
# readings_pdf = readings_df.to_pandas()


In [34]:
# parse the json string, normalize
normalized_pdf = pd.json_normalize(sensor_pdf['V'].apply(json.loads))
normalized_readings_pdf = pd.json_normalize(normalized_pdf['data.data'].explode())

In [35]:
normalized_readings_pdf

,battery_level,center_point_distance,container_id,device_serial_no,fill_level,fill_level_percentage,id,latitude,longitude,raw_reading_distance,...,multi_reading_distances.15,multi_reading_distances.2,multi_reading_distances.3,multi_reading_distances.4,multi_reading_distances.5,multi_reading_distances.6,multi_reading_distances.7,multi_reading_distances.8,multi_reading_distances.9,multi_reading_distances
0,3616.0,814.0,029e8d30-8013-11e9-af1b-910c5bebc890,fe91dba329502cead5714eb9cbcd15eb,NaN,0.0,9a34ca41-91e5-11e9-aad5-811cf201ecdb,NaN,NaN,837.0,...,863.0,808.0,808.0,831.0,848.0,838.0,843.0,815.0,826.0,NaN
1,3618.0,819.0,029e8d30-8013-11e9-af1b-910c5bebc890,fe91dba329502cead5714eb9cbcd15eb,NaN,0.0,9a34ca40-91e5-11e9-951b-54806d619632,NaN,NaN,830.0,...,860.0,811.0,794.0,842.0,849.0,827.0,840.0,802.0,808.0,NaN
2,3618.0,401.0,029e8d30-8013-11e9-af1b-910c5bebc890,fe91dba329502cead5714eb9cbcd15eb,NaN,61.0,9a34a332-91e5-11e9-899b-5697b56c0b14,NaN,NaN,416.0,...,430.0,406.0,391.0,420.0,444.0,393.0,374.0,448.0,442.0,NaN
3,3618.0,397.0,029e8d30-8013-11e9-af1b-910c5bebc890,fe91dba329502cead5714eb9cbcd15eb,NaN,61.0,9a34a331-91e5-11e9-8f1f-dfbb9ae3ead3,NaN,NaN,413.0,...,420.0,396.0,394.0,427.0,443.0,385.0,371.0,442.0,438.0,NaN
4,3618.0,396.0,029e8d30-8013-11e9-af1b-910c5bebc890,fe91dba329502cead5714eb9cbcd15eb,NaN,61.0,9a34a330-91e5-11e9-a3e8-7ba821d55573,NaN,NaN,412.0,...,412.0,399.0,381.0,421.0,445.0,385.0,366.0,446.0,440.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621390,3618.0,482.0,12744540-2d55-11e8-bcc1-309ced4e45f3,2c0282c193a12e319822667929e56590,NaN,49.0,3e711a41-9723-11e9-b37e-fc87766f6b5c,NaN,NaN,493.0,...,488.0,475.0,479.0,523.0,534.0,513.0,494.0,493.0,485.0,NaN
621391,3618.0,474.0,12744540-2d55-11e8-bcc1-309ced4e45f3,2c0282c193a12e319822667929e56590,NaN,50.0,3e711a40-9723-11e9-a8e2-4f04a2b60433,NaN,NaN,488.0,...,479.0,478.0,491.0,509.0,520.0,505.0,490.0,485.0,478.0,NaN
621392,3618.0,472.0,12744540-2d55-11e8-bcc1-309ced4e45f3,2c0282c193a12e319822667929e56590,NaN,51.0,3e70f332-9723-11e9-9c93-828293a40a58,NaN,NaN,484.0,...,471.0,469.0,478.0,510.0,522.0,508.0,499.0,474.0,473.0,NaN
621393,3618.0,561.0,12744540-2d55-11e8-bcc1-309ced4e45f3,2c0282c193a12e319822667929e56590,NaN,37.0,3e70f331-9723-11e9-ace2-a902ab6e99e0,NaN,NaN,574.0,...,589.0,522.0,524.0,580.0,636.0,618.0,580.0,566.0,574.0,NaN


In [59]:
session.write_pandas(normalized_readings_pdf, table_name="TIMESERIES_NORMAL", database="TAKE_HOME_USER2", schema="SENSOR")
# ProgrammingError: 090106 (22000): Cannot perform CREATE TEMPSTAGE. This session does not have a current schema. Call 'USE SCHEMA', or use a qualified name.
# asr_df_final.createOrReplaceView(f"TAKE_HOME_USER2.ASR.ASR_ANALYTICS_2019")

: 

: 

In [4]:
readings_pdf.head()


,ALERT_STATUS,FILL_LEVEL,ID,DATA
0,"""LOW""",0,"""002204d0-8700-11e9-ae2e-7d794ea7467c""","[\n {\n ""battery_level"": 3643,\n ""cente..."
1,"""LOW""",78,"""069bf090-8558-11e9-b4ed-3ed6868acd76""","[\n {\n ""battery_level"": 3622,\n ""cente..."
2,"""LOW""",0,"""09d27f20-8587-11e9-b23b-b0bd4914b555""","[\n {\n ""battery_level"": 3611,\n ""cente..."
3,"""LOW""",0,"""108a6690-8706-11e9-b8c5-79bae3aec1b9""","[\n {\n ""battery_level"": 3584,\n ""cente..."
4,"""LOW""",0,"""12180fa0-848e-11e9-bbcc-e92ac0bf4d30""","[\n {\n ""battery_level"": 3635,\n ""cente..."


In [39]:
# should just use https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html
dummy_list = []
for index, row in readings_pdf[0:10].iterrows():
    readings = json.loads(row['DATA'])
    rolling_count = 0
    # rche_df.loc[index, 'wgs1984_latitude'] = dict_temp['lat']
    # print(readings)
    readings_df = pd.DataFrame.from_dict(readings)
    readings_df.rolling(window=4, min_periods=4)

    alerts = readings_df.rolling(window=4, min_periods=4)['fill_level_percentage']
    print(alerts)
    # services = readings_df.rolling(window=4, min_periods=4)['fill_level_percentage'].mean() < 20
    # print(alerts, services)
    # print(readings_df.head())
    # for reading in readings:
    #     print(reading)
    #     pass


0     False
1     False
2     False
3      True
4      True
      ...  
60    False
61    False
62    False
63    False
64    False
Name: fill_level_percentage, Length: 65, dtype: bool 0     False
1     False
2     False
3     False
4     False
      ...  
60    False
61    False
62    False
63    False
64    False
Name: fill_level_percentage, Length: 65, dtype: bool
0     False
1     False
2     False
3     False
4     False
      ...  
76    False
77    False
78    False
79    False
80    False
Name: fill_level_percentage, Length: 81, dtype: bool 0     False
1     False
2     False
3     False
4     False
      ...  
76    False
77    False
78    False
79    False
80    False
Name: fill_level_percentage, Length: 81, dtype: bool


KeyError: 'Column not found: fill_level_percentage'

In [7]:
readings_pdf.head()

,ALERT_STATUS,FILL_LEVEL,ID,DATA
0,"""HIGH""",113,"""049ab7f0-7a58-11e9-a7d5-79d7eb57b39a""","[\n {\n ""battery_level"": 3603,\n ""cente..."
1,"""LOW""",1,"""0581b640-80df-11e9-94cf-992c2615833b""","[\n {\n ""battery_level"": 3607,\n ""cente..."
2,"""LOW""",0,"""09147e60-9150-11e9-ac04-efcb622dc410""",[]
3,"""LOW""",0,"""0aeedbe0-80b6-11e9-8024-b1a347456020""","[\n {\n ""battery_level"": 3643,\n ""cente..."
4,"""LOW""",0,"""1044e8e0-9219-11e9-ae3b-42cf2a505c87""",[]


In [28]:
pd.json_normalize(sensor_pdf['V'])

""
0
1
2
3
4
...
6050
6051
6052
6053


In [5]:
# Load the JSON file into a Python dictionary

# Extract the relevant data
container_id = []
fill_level_percentage = []
timestamp = []

for reading in data['data']['data']:
    container_id.append(reading['container_id'])
    fill_level_percentage.append(reading['fill_level_percentage'])
    timestamp.append(datetime.strptime(reading['timestamp'], '%Y-%m-%d %H:%M:%S.%f'))

# Create a DataFrame from the extracted data
df = pd.DataFrame({
    'container_id': container_id,
    'fill_level_percentage': fill_level_percentage,
    'timestamp': timestamp
})

NameError: name 'data' is not defined

In [ ]:
# Identify when an alert is triggered and when a can is serviced
alerts = df.groupby('container_id').rolling(window=4, min_periods=4)['fill_level_percentage'].mean() > 80
services = df.groupby('container_id').rolling(window=4, min_periods=4)['fill_level_percentage'].mean() < 20

# Calculate the time taken for the garbage can to be serviced after an alert was sent to the waste management company
df['alert_triggered'] = alerts.groupby('container_id').transform(lambda x: x[x].index[0])
df['serviced'] = services.groupby('container_id').transform(lambda x: x[x].index[0])
df['pick_up_time'] = (df['serviced'] - df['alert_triggered']).dt.total_seconds() / 3600

# Determine if a can has overflowed
df['overflow'] = (df['fill_level_percentage'] > 100).rolling(window=3, min_periods=3).sum()
df['overflow'] = df.groupby('container_id')['overflow'].shift(-2)

# Filter out duplicate readings and missing time periods
df = df.drop_duplicates(subset=['container_id', 'timestamp']).set_index('timestamp')
df = df.groupby('container_id').resample('5min').ffill()

# Select the relevant columns and reset the index
df = df[['container_id', 'alert_triggered', 'serviced', 'pick_up_time', 'overflow']].reset_index()

# Convert the timestamps to the required format
df['alert_triggered'] = df['alert_triggered'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
df['serviced'] = df['serviced'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')

# Display the first few rows of the final dataset
print(df.head())

In [29]:
normalized_pdf

,canHasToken,status,data.container.address_city,data.container.address_country,data.container.address_first_line,data.container.address_post_code,data.container.address_second_line,data.container.alert_status,data.container.alert_status_at,data.container.assignment_id,...,data.container.route_profile,data.container.service_detection_mode,data.container.serviced_now,data.container.should_forward_report,data.container.status,data.container.status_at,data.container.tenant,data.container.updated_at,data.container.updated_by,data.data
0,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ2YWx1Z...,accepted,San Francisco,United States of America,Market Street,94111,,LOW,1560872661000,None,...,DRIVING,DATA_OR_ACCELEROMETER,None,False,ACTIVE,None,sf,1560875093424,11111111-1111-1111-1111-111111111111,"[{'battery_level': 3616, 'center_point_distanc..."
1,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ2YWx1Z...,accepted,San Francisco,United States of America,Haight Street,94117,,LOW,1560366464000,None,...,DRIVING,DATA_OR_ACCELEROMETER,None,False,ACTIVE,None,sf,1560585600367,11111111-1111-1111-1111-111111111111,[]
2,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ2YWx1Z...,accepted,San Francisco,United States of America,3rd Street,94124,,LOW,None,None,...,DRIVING,DATA_OR_ACCELEROMETER,None,False,ACTIVE,None,sf,1560805700392,727ea050-afb6-11e8-869c-fb38af225928,[]
3,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ2YWx1Z...,accepted,San Francisco,United States of America,Market Street,94104,,LOW,None,None,...,DRIVING,DATA_OR_ACCELEROMETER,None,False,ACTIVE,None,sf,1559088000473,11111111-1111-1111-1111-111111111111,[]
4,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ2YWx1Z...,accepted,San Francisco,United States of America,Stockton Street,94108,,LOW,1560907802000,None,...,DRIVING,DATA_OR_ACCELEROMETER,None,False,ACTIVE,None,sf,1560908073789,11111111-1111-1111-1111-111111111111,"[{'battery_level': 3637, 'center_point_distanc..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6050,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ2YWx1Z...,accepted,San Francisco,United States of America,Geary Boulevard,94118,,HIGH,1561606105000,None,...,DRIVING,DATA_OR_ACCELEROMETER,None,False,ACTIVE,None,sf,1561165192047,11111111-1111-1111-1111-111111111111,"[{'battery_level': 3618, 'center_point_distanc..."
6051,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ2YWx1Z...,accepted,San Francisco,United States of America,3rd Street,94124,,HIGH,1561707158000,None,...,DRIVING,DATA_OR_ACCELEROMETER,None,False,ACTIVE,None,sf,1561707799467,11111111-1111-1111-1111-111111111111,"[{'battery_level': 0, 'center_point_distance':..."
6052,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ2YWx1Z...,accepted,San Francisco,United States of America,California Street,94115,,LOW,1561462319000,None,...,DRIVING,DATA_OR_ACCELEROMETER,None,False,ACTIVE,None,sf,1561680001518,11111111-1111-1111-1111-111111111111,"[{'battery_level': 3622, 'center_point_distanc..."
6053,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ2YWx1Z...,accepted,San Francisco,United States of America,Polk Street,94109,,LOW,1561670095000,None,...,DRIVING,DATA_OR_ACCELEROMETER,None,False,ACTIVE,None,sf,1561672573455,11111111-1111-1111-1111-111111111111,"[{'battery_level': 3628, 'center_point_distanc..."


,battery_level,center_point_distance,container_id,device_serial_no,fill_level,fill_level_percentage,id,latitude,longitude,raw_reading_distance,...,multi_reading_distances.15,multi_reading_distances.2,multi_reading_distances.3,multi_reading_distances.4,multi_reading_distances.5,multi_reading_distances.6,multi_reading_distances.7,multi_reading_distances.8,multi_reading_distances.9,multi_reading_distances
0,3616.0,814.0,029e8d30-8013-11e9-af1b-910c5bebc890,fe91dba329502cead5714eb9cbcd15eb,NaN,0.0,9a34ca41-91e5-11e9-aad5-811cf201ecdb,NaN,NaN,837.0,...,863.0,808.0,808.0,831.0,848.0,838.0,843.0,815.0,826.0,NaN
1,3618.0,819.0,029e8d30-8013-11e9-af1b-910c5bebc890,fe91dba329502cead5714eb9cbcd15eb,NaN,0.0,9a34ca40-91e5-11e9-951b-54806d619632,NaN,NaN,830.0,...,860.0,811.0,794.0,842.0,849.0,827.0,840.0,802.0,808.0,NaN
2,3618.0,401.0,029e8d30-8013-11e9-af1b-910c5bebc890,fe91dba329502cead5714eb9cbcd15eb,NaN,61.0,9a34a332-91e5-11e9-899b-5697b56c0b14,NaN,NaN,416.0,...,430.0,406.0,391.0,420.0,444.0,393.0,374.0,448.0,442.0,NaN
3,3618.0,397.0,029e8d30-8013-11e9-af1b-910c5bebc890,fe91dba329502cead5714eb9cbcd15eb,NaN,61.0,9a34a331-91e5-11e9-8f1f-dfbb9ae3ead3,NaN,NaN,413.0,...,420.0,396.0,394.0,427.0,443.0,385.0,371.0,442.0,438.0,NaN
4,3618.0,396.0,029e8d30-8013-11e9-af1b-910c5bebc890,fe91dba329502cead5714eb9cbcd15eb,NaN,61.0,9a34a330-91e5-11e9-a3e8-7ba821d55573,NaN,NaN,412.0,...,412.0,399.0,381.0,421.0,445.0,385.0,366.0,446.0,440.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621390,3618.0,482.0,12744540-2d55-11e8-bcc1-309ced4e45f3,2c0282c193a12e319822667929e56590,NaN,49.0,3e711a41-9723-11e9-b37e-fc87766f6b5c,NaN,NaN,493.0,...,488.0,475.0,479.0,523.0,534.0,513.0,494.0,493.0,485.0,NaN
621391,3618.0,474.0,12744540-2d55-11e8-bcc1-309ced4e45f3,2c0282c193a12e319822667929e56590,NaN,50.0,3e711a40-9723-11e9-a8e2-4f04a2b60433,NaN,NaN,488.0,...,479.0,478.0,491.0,509.0,520.0,505.0,490.0,485.0,478.0,NaN
621392,3618.0,472.0,12744540-2d55-11e8-bcc1-309ced4e45f3,2c0282c193a12e319822667929e56590,NaN,51.0,3e70f332-9723-11e9-9c93-828293a40a58,NaN,NaN,484.0,...,471.0,469.0,478.0,510.0,522.0,508.0,499.0,474.0,473.0,NaN
621393,3618.0,561.0,12744540-2d55-11e8-bcc1-309ced4e45f3,2c0282c193a12e319822667929e56590,NaN,37.0,3e70f331-9723-11e9-ace2-a902ab6e99e0,NaN,NaN,574.0,...,589.0,522.0,524.0,580.0,636.0,618.0,580.0,566.0,574.0,NaN
